In [1]:
import autograd
import autograd.numpy as np
from autograd import grad
from autograd import checkpoint
from autograd.extend import primitive

import numpy as onp

%load_ext memory_profiler

from builtins import range, list as ag_list, tuple as ag_tuple
import autograd.numpy as np
import autograd.numpy.random as npr
from autograd import grad
from autograd.scipy.misc import logsumexp
from os.path import dirname, join
from autograd.misc.optimizers import adam


### Helper functions #################

def sigmoid(x):
    return 0.5*(np.tanh(x) + 1.0)   # Output ranges from 0 to 1.

In [2]:
def create_rnn_params(input_size, state_size, output_size,
                      param_scale=0.01, rs=npr.RandomState(0)):
    return {'change': rs.randn(input_size + state_size + 1, state_size) * param_scale,
            'predict': rs.randn(state_size + 1, output_size) * param_scale,
            'init hiddens': rs.randn(1, state_size) * param_scale,}

def hiddens_to_output_probs(theta, hiddens):
    output = concat_and_multiply(theta['predict'], hiddens)
    return output - logsumexp(output, axis=1, keepdims=True)

def concat_and_multiply(weights, *args):
    cat_state = np.hstack(args + (np.ones((args[0].shape[0], 1)),))
    return np.dot(cat_state, weights)

input_size = 10
state_size = 15
output_size = 15

batch_size = 1
seq_len = 400
num_checkpoints = 20

theta = create_rnn_params(input_size, state_size, output_size)

np.random.seed(0)
inputs = np.random.randn(seq_len, batch_size, input_size)
hidden_single = npr.RandomState(0).randn(1, state_size) * .01
hidden = np.repeat(hidden_single, batch_size, axis=0)
output = hiddens_to_output_probs(theta, hidden)

init_ch = ag_list([hidden, output])

In [3]:
# My Code
from autograd.differential_operators import binomial_checkpoint
from autograd.builtins import list as ag_list, tuple as ag_tuple

def rnn(theta, init_ch, x):
    def update_rnn(x, hiddens):
        return np.tanh(concat_and_multiply(theta['change'], x, hiddens))
    init_cell = init_ch[0]
    init_hidden = init_ch[1]
    
    cell = update_rnn(x, init_cell)
    hidden = hiddens_to_output_probs(theta, cell)
    
    return ag_tuple((cell, hidden))

In [12]:
loop, unwrapped_loop = binomial_checkpoint(rnn, seq_len, num_checkpoints)

def rnn_predict(params, inputs):
    num_sequences = inputs.shape[1]
    
    hidden_single = npr.RandomState(0).randn(1, state_size) * .01
    hidden = np.repeat(hidden_single, num_sequences, axis=0)
    output = hiddens_to_output_probs(params, hidden)
        
    outputs = loop(params, ag_tuple((hidden, output)), inputs)
    return outputs

In [13]:
from time import time

loop, unwrapped_loop = binomial_checkpoint(rnn, seq_len, num_checkpoints)

hidden_single = npr.RandomState(0).randn(1, state_size) * .01    
hidden = np.repeat(hidden_single, batch_size, axis=0)

f = lambda theta: rnn_predict(theta, inputs)
g = lambda theta: np.sum(sum(h for c, h in f(theta)))

x1 = g(theta)

start = time()
%memit g1 = grad(g)(theta)
end = time()

print("elapsed time: ", end - start)

peak memory: 80.45 MiB, increment: 0.13 MiB
elapsed time:  3.2334201335906982


In [10]:
def rnn_predict(params, inputs):
    def update_rnn(input, hiddens):
        return np.tanh(concat_and_multiply(params['change'], input, hiddens))
    
    num_sequences = inputs.shape[1]
    
    hidden_single = npr.RandomState(0).randn(1, state_size) * .01
    hidden = np.repeat(hidden_single, num_sequences, axis=0)  
    output = hiddens_to_output_probs(params, hidden)
    
    outputs = [output]

    for input in inputs:  # Iterate over time steps.
        hidden, output = rnn(params, (hidden, output), input)
        outputs.append(output)
    
    return outputs

In [11]:
f = lambda theta: rnn_predict(theta, inputs)
g = lambda theta: np.sum(f(theta))

x2 = g(theta)

start = time()
%memit g2 = grad(g)(theta)
end = time()

print("elapsed time: ", end - start)

peak memory: 90.55 MiB, increment: 10.88 MiB
elapsed time:  1.058408260345459


In [8]:
# check correctness of gradients

print(x1 - x2)

for key in g1:
    print(key, np.sum((g1[key] - g2[key])**2))

-3.637978807091713e-12
change 1.2061753316069602e-31
predict 5.360460534159286e-29
init hiddens 0.0
